<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Trans-Fun" data-toc-modified-id="Trans-Fun-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Trans Fun</a></span></li><li><span><a href="#My-First-Transform" data-toc-modified-id="My-First-Transform-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>My First Transform</a></span></li><li><span><a href="#The-@Transform-decorator" data-toc-modified-id="The-@Transform-decorator-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The <code>@Transform</code> decorator</a></span></li><li><span><a href="#Type-annotations" data-toc-modified-id="Type-annotations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Type annotations</a></span></li><li><span><a href="#WTFunc" data-toc-modified-id="WTFunc-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>WTFunc</a></span></li></ul></div>

In [ ]:
#export
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.utils import *
from fastcore.dispatch import *
from fastcore.test import *
from fastcore.transform import *
import torch

# Trans Fun
> Re-implementation of class Transform.

Questions:
- How will my trans handle tuples?
- Will I need to do any preprocessing in `setup`?
- Does it need to handle valid differently than train?
- What order does it enter in the entire pipeline?
- Can I just patch an existing transform for a certain type by creating `encodes` and `decodes` methods for new types?

# My First Transform

##### Add `encodes` using the `@` decorator to an existing `Transform`

In [ ]:
class C(Transform): pass

In [ ]:
C()

C:
encodes: decodes: 

In [ ]:
@C
def encodes(self, x): return x+1
f = C()
f

C:
encodes: (object,object) -> encodes
decodes: 

In [ ]:
x,y,z = 1,2,3
xy = 1,2
xyz = 1,2,3

In [ ]:
f(x)

2

In [ ]:
f(xy)

(2, 3)

In [ ]:
f((0,0,0,0,0,0,0))

(1, 1, 1, 1, 1, 1, 1)

##### Add `decodes` with `@`

In [ ]:
class D(C): pass
D

__main__.D

In [ ]:
D()

D:
encodes: (object,object) -> encodes
decodes: 

In [ ]:
@D
def decodes(self, x): return x-1

In [ ]:
D()

D:
encodes: (object,object) -> encodes
decodes: (object,object) -> decodes

In [ ]:
g = D()

In [ ]:
g(x),g(y),g(z)

(2, 3, 4)

In [ ]:
g(xyz)

(2, 3, 4)

In [ ]:
g((1,2,3))

(2, 3, 4)

In [ ]:
g.decodes(x)

0

In [ ]:
g.decodes(xy)

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

Notes: `decodes` does not handle tuples automatically (`encodes` does).

# The `@Transform` decorator

In [ ]:
# Boring:
f = Transform(lambda x: x**2)
f(2)

4

In [ ]:
f

<lambda>:
encodes: (object,object) -> <lambda>decodes: 

In [ ]:
# Exciting!:
@Transform
def g(x): return x**2
g(2)

4

In [ ]:
g

g:
encodes: (object,object) -> gdecodes: 

The `@Transform` decorator is awesome for quickly turning functions into Transforms that only encode.

# Type annotations

Multiple encodes for different types:

In [ ]:
class T(Transform): pass
@T
def encodes(self,x:int): return x-1
@T
def encodes(self,x:float): return x+1
f = T()
x = f(1)
print(x, type(x))
y = f(1.)
print(y, type(y))

0 <class 'int'>
2.0 <class 'float'>


This next one has *return type annotations*, which cast the output as the return type.

In [ ]:
class T(Transform):
    def encodes(self,x:(int,float)): return x-1
    def encodes(self,x:(float,int)): return x+1

f = T()
x = f(1)
print(x, type(x))
y = f(1.)
print(y, type(y))

2 <class 'int'>
2.0 <class 'float'>


In [ ]:
def func(x): return Int(x+1)
def dec (x): return x-1
f = Transform(func,dec)
t = f(1);
u = f.decode(f(1))
test_eq(1,u)
test_eq_type(t, Int(2))
test_eq_type(f.decode(t), Int(1))

In [ ]:
test_eq_type?  # test for == AND same type

In [ ]:
class T(Transform):
    def encodes(self, xy): x,y=xy; return [x+y, y]
    def decodes(self, xy): x,y=xy; return [x-y, y]
f=T()
x=[1,2]
f(x)

[3, 2]

If the transform as a `split_idx` then it's only applied if split_idx param matches.

In [ ]:
f.split_idx = 1

In [ ]:
f(x)

[1, 2]

In [ ]:
f(x, split_idx=0)

[1, 2]

In [ ]:
f(x, split_idx=1)

[3, 2]

# WTFunc

In [ ]:
f = Func('sqrt')(math)
f(4)

2.0